In [1]:
import sqlite3 as sql
import pandas as pd

### Top Lenders

In [2]:
topLenders = """
SELECT year, Table1.lei, respondent_name, SUM(amount) AS "total", COUNT(*) AS loans
FROM 

(SELECT "Activity Year" AS year, "Legal Entity Identifier (LEI)" AS lei, "Occupancy Type" AS occupancy,
"Loan Type" AS loan_type, "Loan Amount" AS amount, "Interest Rate" AS rate, "Census Tract" AS census_tract,
Income AS income, "Rate Spread" AS rate_spread, "Total Loan Costs" AS loan_costs,
"Origination Charges" AS origination_charge, "Discount Points" AS discount_points,
"Lender Credits" AS lender_credits, "Loan Term" AS term, "Property Value" AS prop_value,
"Total Units" AS units
FROM MLAR 
WHERE "Loan Purpose" = 1 -- loan was for purchase 
AND "Action Taken" = 1 -- loan was approved
AND "Business or Commercial Purpose" != 1 -- loan not for business purpose
AND "Reverse Mortgage" != 1 -- not a reverse mortgage
AND "Open-End Line of Credit" != 1 -- not open-end line of credit
AND "Total Units" IN (1, 2, 3, 4) -- not multifamily
) AS Table1
LEFT JOIN Institutions
ON Table1.lei = Institutions.lei

GROUP BY year, Table1.lei
ORDER BY "total" DESC, loans DESC
LIMIT 100
"""

In [3]:
with sql.connect('./cleaned/hmda.db') as connection:
    tl = pd.read_sql(topLenders, connection)
tl.head()

,year,lei,respondent_name,total,loans
0,2023,549300HW662MN1WU8550,United Wholesale Mortgage,89984280000,230790
1,2022,549300HW662MN1WU8550,United Wholesale Mortgage,85714715000,220379
2,2022,549300FGXN1K3HLB1R50,ROCKET MORTGAGE,51829020000,155136
3,2023,549300FGXN1K3HLB1R50,ROCKET MORTGAGE,44795750000,129872
4,2022,KB1H1DSPRFMYMCUFXT09,WELLS FARGO BANK NA,40248005000,67945


In [4]:
byTotal = tl.loc[tl.total.isin(tl.groupby('year').total.nlargest(10)), ['year', 'respondent_name', 'total']].sort_values(
    by=['year', 'total'], ascending=[True, False])
byTotal['total'] = round(byTotal.total / 1000000000, 2)
byTotal['row_filter'] = 'Total (billion $)'
byTotal.rename(columns={'year':'Year', 'respondent_name':'Originator', 'total':'Value'}, inplace=True)

byLoans = tl.loc[tl.loans.isin(tl.groupby('year').loans.nlargest(10)), ['year', 'respondent_name', 'loans']].sort_values(
    by=['year', 'loans'], ascending=[True, False])
byLoans['loans'] = round(byLoans.loans / 1000, 2)
byLoans['row_filter'] = 'Loans (thousands)'
byLoans.rename(columns={'year':'Year', 'respondent_name':'Originator', 'loans':'Value'}, inplace=True)

new_name = {
    'Bank of America NA':'Bank of America',
    'CALIBER HOME LOANS, INC.':'Caliber',
    'CROSSCOUNTRY MORTGAGE, LLC':'CrossCountry',
    'DHI Mortgage Company Limited':'DHI',
    'FAIRWAY INDEPENDENT MORT CORP':'Fairway',
    'GUARANTEED RATE, INC':'Guaranteed Rate',
    'JPMorgan Chase Bank, NA':'JPMorgan Chase',
    'LOANDEPOT.COM, LLC':'LoanDepot',
    'Lennar Mortgage, LLC':'Lennar',
    'MOVEMENT MORTGAGE, LLC':'Movement Mortgage',
    'Mortgage Research Center':'Mortgage Research Center',
    'ROCKET MORTGAGE':'Rocket',
    'US BANK, N.A.':'US Bank',
    'United Wholesale Mortgage':'UWM',
    'WELLS FARGO BANK NA':'Wells Fargo'
}

tl = pd.concat([byTotal, byLoans], ignore_index=True)
tl['Originator'] = [new_name[x] for x in tl.Originator]
tl.to_csv('./Flourish/TopLenders.csv', index=False)

### County-level

In [5]:
monthlyPayment = """SELECT "Activity Year" AS year, SUBSTR("Census Tract",1,2) AS state, SUBSTR("Census Tract",3,3) AS county, 
"Loan Amount" AS amount, "Interest Rate" AS rate, Income/12 AS monthly_income 
FROM MLAR 
WHERE "Loan Purpose" = 1 -- loan was for purhchase 
AND "Action Taken" = 1 -- loan was approved
AND "Business or Commercial Purpose" != 1 -- loan not for business purpose
AND "Reverse Mortgage" != 1 -- not a reverse mortgage
AND "Open-End Line of Credit" != 1 -- not open-end line of credit
AND "Total Units" IN (1, 2, 3, 4) -- not multifamily
--special
AND state NOTNULL AND county NOTNULL 
AND "Loan Term" = 360
"""

In [6]:
with sql.connect('./cleaned/hmda.db') as connection:
    mp = pd.read_sql(monthlyPayment, connection)
mp.head()

,year,state,county,amount,rate,monthly_income
0,2022,29,071,285000,7.000,9.0
1,2022,55,025,605000,4.875,10.0
2,2022,55,111,345000,5.875,10.0
3,2022,12,011,165000,8.625,6.0
4,2022,01,003,1125000,5.000,27.0


In [7]:
# keep only counties with at least 100 originations in both years
mp['state_county'] = mp.state + mp.county
counts = mp.groupby(['year','state_county']).amount.count().reset_index()
state_counties = counts.loc[counts.amount>=100, 'state_county']
state_counties = pd.DataFrame(state_counties.value_counts())

mp = mp.loc[mp.state_county.isin(state_counties.loc[state_counties['count']==2].index)]

In [8]:
mp['monthly_payment'] = (mp.rate/100/12*mp.amount)/(1 - ((1 + (mp.rate/100/12)) ** (-360)))
mp['perc_income'] = mp.monthly_payment / (mp.monthly_income * 1000) * 100

In [9]:
mp_grouped = mp.groupby(['year', 'state', 'county']).agg({
    'state_county':'count',
    'rate':'median',
    'monthly_payment':'median',
    'perc_income':'median'
})
mp_grouped['perc_over_28'] = mp.loc[mp.perc_income>28].groupby(['year','state','county']).perc_income.count().div(
    mp.groupby(['year', 'state', 'county']).perc_income.count()
)
mp_grouped = mp_grouped.reset_index().rename(columns={
    'year':'Year',
    'state_county':'Loans',
    'rate':'Median Rate',
    'monthly_payment':'Median Monthly Payment',
    'perc_income':'Median % of Income',
    'perc_over_28':'% of Recipients Paying >28% of Income'
})
mp_grouped['state_county'] = mp_grouped.state + mp_grouped.county

mp_grouped['% of Recipients Paying >28% of Income'] = round(mp_grouped['% of Recipients Paying >28% of Income'].mul(100))
mp_grouped['Median Rate'] = round(mp_grouped['Median Rate'],1)
mp_grouped['Median % of Income'] = round(mp_grouped['Median % of Income'],1)
mp_grouped['Median Monthly Payment'] = mp_grouped['Median Monthly Payment'].astype('int')

# add county names and fips code for geo matching
counties = pd.read_excel('./cleaned/CountyNames.xlsx', dtype='str')
mp_grouped = mp_grouped.merge(counties, on='state_county', how='left')
mp_grouped.rename(columns={'NAME':'County'}, inplace=True)
mp_grouped.drop(columns=['state','county', 'STATEFP', 'COUNTYFP'], inplace=True)
mp_grouped

,Year,Loans,Median Rate,Median Monthly Payment,Median % of Income,% of Recipients Paying >28% of Income,state_county,County
0,2022,955,5.0,1315,20.6,21.0,01001,Autauga
1,2022,5456,4.8,1508,20.0,22.0,01003,Baldwin
2,2022,507,5.2,1099,20.5,24.0,01009,Blount
3,2022,1083,5.1,889,20.2,21.0,01015,Calhoun
4,2022,214,5.2,827,19.2,18.0,01017,Chambers
...,...,...,...,...,...,...,...,...
3975,2023,278,6.2,831,23.1,31.0,72135,Toa Alta
3976,2023,322,6.2,765,22.5,30.0,72137,Toa Baja
3977,2023,329,6.5,756,21.8,25.0,72139,Trujillo Alto
3978,2023,135,6.5,739,20.5,25.0,72143,Vega Alta


In [10]:
mp_grouped.loc[(mp_grouped.Year==2023)&(mp_grouped['% of Recipients Paying >28% of Income']>28)]

,Year,Loans,Median Rate,Median Monthly Payment,Median % of Income,% of Recipients Paying >28% of Income,state_county,County
1993,2023,837,6.8,1135,22.7,30.0,01015,Calhoun
1996,2023,328,6.6,1295,22.9,30.0,01021,Chilton
2003,2023,877,6.8,1589,22.7,29.0,01051,Elmore
2005,2023,783,6.6,1164,23.0,30.0,01055,Etowah
2010,2023,308,6.8,1242,22.1,30.0,01071,Jackson
...,...,...,...,...,...,...,...,...
3969,2023,176,6.2,846,22.3,29.0,72085,Las Piedras
3972,2023,377,6.5,616,21.6,32.0,72113,Ponce
3975,2023,278,6.2,831,23.1,31.0,72135,Toa Alta
3976,2023,322,6.2,765,22.5,30.0,72137,Toa Baja


In [11]:
mp_grouped.loc[(mp_grouped.Year==2023)&(mp_grouped.Loans>=100)].to_excel('./Flourish/2023_CountyLevel.xlsx', index=False)

In [12]:
loans_change = mp_grouped[['Year', 'Loans', 'state_county']].pivot(index='state_county', columns='Year', values='Loans')
loans_change['change'] = loans_change[2023] - loans_change[2022]
loans_change['fell'] = [x < 0 for x in loans_change['change']]
loans_change.fell.value_counts()

fell
True     1908
False      82
Name: count, dtype: int64

In [13]:
rate_rose = mp_grouped[['Year', 'Median Rate', 'state_county']].pivot(index='state_county', columns='Year', values='Median Rate')
rate_rose['change'] = rate_rose[2023] - rate_rose[2022]
rate_rose['rose'] = [x > 0 for x in rate_rose['change']]
rate_rose.rose.value_counts()

rose
True    1990
Name: count, dtype: int64

In [14]:
unaffordable_rose = mp_grouped[['Year', '% of Recipients Paying >28% of Income', 'state_county']].pivot(index='state_county', columns='Year', values='% of Recipients Paying >28% of Income')
unaffordable_rose['change'] = unaffordable_rose[2023] - unaffordable_rose[2022]
unaffordable_rose['rose'] = [x > 0 for x in unaffordable_rose['change']]
unaffordable_rose.rose.value_counts()

rose
True     1887
False     103
Name: count, dtype: int64

### Overall change

In [15]:
statsByYear = """
SELECT "Activity Year" AS year, 
"Loan Amount" AS amount, "Interest Rate" AS rate, Income/12*1000 AS monthly_income 
FROM MLAR 
WHERE "Loan Purpose" = 1 -- loan was for purchase 
AND "Action Taken" = 1 -- loan was approved
AND "Business or Commercial Purpose" != 1 -- loan not for business purpose
AND "Reverse Mortgage" != 1 -- not a reverse mortgage
AND "Open-End Line of Credit" != 1 -- not open-end line of credit
AND "Total Units" IN (1, 2, 3, 4) -- not multifamily
--special
AND "Loan Term" = 360
"""

In [16]:
with sql.connect('./cleaned/hmda.db') as connection:
    byYear = pd.read_sql(statsByYear, connection)
byYear.head()

,year,amount,rate,monthly_income
0,2022,285000,7.000,9000.0
1,2022,605000,4.875,10000.0
2,2022,345000,5.875,10000.0
3,2022,165000,8.625,6000.0
4,2022,1125000,5.000,27000.0


In [17]:
byYear['monthly_payment'] = (byYear.rate/100/12*byYear.amount)/(1 - ((1 + (byYear.rate/100/12)) ** (-360)))
byYear['perc_income'] = byYear.monthly_payment / (byYear.monthly_income) * 100

byYear[['year', 'rate', 'monthly_payment']].groupby('year').median()

,rate,monthly_payment
year,,
2022,4.990,1672.605613
2023,6.625,1981.651507


In [18]:
rates = byYear.dropna(subset=['rate'])[['year', 'rate']]
rates = rates.loc[rates.rate>0]
rates.loc[rates.rate>50, 'rate'] = rates.loc[rates.rate>50, 'rate'] / 10

In [19]:
bins = list(range(1,60))
bins = [float(x)/4 for x in bins]

rates['bin'] = pd.cut(rates.rate, bins=bins)

In [20]:
rates.groupby(['year', 'bin']).count().reset_index().to_csv('./Flourish/RateDistribution.csv', index=False)

In [21]:
byYear.dropna(how='any', inplace=True)

In [22]:
share_income = byYear.copy()[['year', 'perc_income']]
share_income['bin'] = pd.cut(share_income.perc_income, list(range(0, 100, 5)))
share_income = share_income.groupby(['year', 'bin']).count().reset_index()
share_income['bin'] = [str(x).split(', ')[1].rstrip(']') for x in share_income['bin']]
share_income.to_csv('./Flourish/PercIncomeDistribution.csv', index=False)

In [23]:
notOnly30 = """
SELECT "Activity Year" AS year, 
"Loan Amount" AS amount, "Interest Rate" AS rate, Income/12*1000 AS monthly_income 
FROM MLAR 
WHERE "Loan Purpose" = 1 -- loan was for purchase 
AND "Action Taken" = 1 -- loan was approved
AND "Business or Commercial Purpose" != 1 -- loan not for business purpose
AND "Reverse Mortgage" != 1 -- not a reverse mortgage
AND "Open-End Line of Credit" != 1 -- not open-end line of credit
AND "Total Units" IN (1, 2, 3, 4) -- not multifamily
"""

In [24]:
with sql.connect('./cleaned/hmda.db') as connection:
    byYearNotOnly30 = pd.read_sql(notOnly30, connection)
byYearNotOnly30.head()

,year,amount,rate,monthly_income
0,2022,285000,7.000,9000.0
1,2022,265000,6.875,5000.0
2,2022,605000,4.875,10000.0
3,2022,345000,5.875,10000.0
4,2022,165000,8.625,6000.0


In [25]:
summ = byYearNotOnly30[['year', 'amount']].groupby('year').sum().merge(
    byYearNotOnly30[['year', 'amount']].groupby('year').count().rename(columns={'amount':'loans'}),
    left_index=True, right_index=True
)
summ['amount'] = round(summ['amount'] / 1000000000000, 2)
summ

,amount,loans
year,,
2022,1.48,4028915
2023,1.14,3193534
